last version cae 26.04.25

In [ ]:
import os
import glob
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report
from tensorflow.keras.saving import register_keras_serializable

# ------------------------------------------------------------
# 0) GPU setup (optional)
# ------------------------------------------------------------
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

# ------------------------------------------------------------
# 1) Define datasets & CSV→TFRecord mapping
# ------------------------------------------------------------
datasets = ['DoS','Fuzzy','RPM','gear','parsed_dataset']
csv_map   = {
    'DoS':            'DoS_dataset.csv',
    'Fuzzy':          'Fuzzy_dataset.csv',
    'RPM':            'RPM_dataset.csv',
    'gear':           'gear_dataset.csv',
    'parsed_dataset': 'parsed_dataset.csv'
}

# build the full list of expected TFRecord files
expected = []
for a in datasets:
    for phase in ('train','val','test'):
        expected.append(f"{a}_{phase}.tfrecord")
        if a != 'parsed_dataset':
            expected.append(f"Normal_{a}_{phase}.tfrecord")

# ------------------------------------------------------------
# 2) Pre‐flight: if any missing → rebuild them, else skip
# ------------------------------------------------------------
if all(os.path.exists(fn) for fn in expected):
    print("✅ All TFRecords found, skipping CSV→TFRecord.")
else:
    # remove any leftovers
    for fn in glob.glob("*_train.tfrecord") + glob.glob("*_val.tfrecord") + glob.glob("*_test.tfrecord"):
        os.remove(fn)
    print("🔄 Rebuilding TFRecords from CSV...")

    # helper fns
    attributes = ['Timestamp','canID','DLC'] + [f'Data{i}' for i in range(8)] + ['Flag']
    def fill_flag(row):
        if not isinstance(row['Flag'], str):
            col = 'Data'+str(int(row['DLC']))
            row['Flag'] = row.get(col, row['Flag'])
        return row
    def convert_canid_bits(cid):
        try:
            b = bin(int(str(cid),16))[2:].zfill(29)
            return np.array(list(map(int,b)),dtype=np.uint8)
        except:
            return np.zeros(29,dtype=np.uint8)
    def hex_to_int(x):
        try:    return int(str(x).strip(),16)
        except:
            try: return int(x)
            except: return 0

    def preprocess_windows(csv_file):
        df = pd.read_csv(csv_file, header=None, names=attributes, low_memory=False)
        df['Timestamp'] = pd.to_numeric(df['Timestamp'], errors='coerce')
        df['DLC']       = pd.to_numeric(df['DLC'], errors='coerce').fillna(0).astype(int)
        df = df.dropna(subset=['Timestamp','canID']).apply(fill_flag, axis=1)
        for i in range(8):
            df[f'Data{i}'] = df[f'Data{i}'].apply(hex_to_int).astype(np.uint8)
        df['Flag'] = df['Flag'].astype(str).str.upper().eq('T').astype(np.uint8)
        df['canBits'] = df['canID'].apply(convert_canid_bits)
        df = df.sort_values('Timestamp')

        bits_all   = np.stack(df['canBits'].values)
        flags_all  = df['Flag'].values
        data_bytes = df[[f'Data{i}' for i in range(8)]].values

        win   = 29
        num_w = bits_all.shape[0] // win

        bits_trunc  = bits_all[:num_w*win].reshape(num_w,win,29)
        flags_trunc = flags_all[:num_w*win].reshape(num_w,win)
        data_trunc  = data_bytes[:num_w*win].reshape(num_w,win,8)

        last_bytes = data_trunc[:,-1,:]
        bits64     = np.unpackbits(last_bytes,axis=1).reshape(-1,8,8)
        data_imgs  = [cv2.resize(b.astype(np.float32),(29,29),
                                  interpolation=cv2.INTER_NEAREST)
                      for b in bits64]

        rows=[]
        for i in range(num_w):
            id_img   = bits_trunc[i].astype(np.uint8)
            data_img = (data_imgs[i]>0.5).astype(np.uint8)
            two_ch   = np.stack([id_img,data_img],axis=-1)
            feat_int = two_ch.flatten().astype(np.int64).tolist()
            lbl      = int(flags_trunc[i].any())
            rows.append({'features':feat_int,'label':lbl})

        out = pd.DataFrame(rows)
        print(f"{csv_file}: #Normal={(out.label==0).sum()}, #Attack={(out.label==1).sum()}")
        return out

    def serialize_example(x,y):
        return tf.train.Example(features=tf.train.Features(feature={
            'input_features': tf.train.Feature(int64_list=tf.train.Int64List(value=x)),
            'label':          tf.train.Feature(int64_list=tf.train.Int64List(value=[y]))
        })).SerializeToString()

    def split_and_write(df, base):
        tfname = f"{base}.tfrecord"
        with tf.io.TFRecordWriter(tfname) as w:
            for _,r in tqdm(df.iterrows(), total=len(df), desc=base):
                w.write(serialize_example(r.features,r.label))
        recs = list(tf.data.TFRecordDataset([tfname]).as_numpy_iterator())
        np.random.shuffle(recs)
        n   = len(recs)
        ntr = int(0.7*n); nvl = int(0.15*n)
        for phase, chunk in zip(['train','val','test'],
                                [recs[:ntr], recs[ntr:ntr+nvl], recs[ntr+nvl:]]):
            out = f"{base}_{phase}.tfrecord"
            with tf.io.TFRecordWriter(out) as w:
                for r in chunk: w.write(r)
            print(f"  → {out}: {len(chunk)} records")

    # loop datasets
    for a in datasets:
        src = csv_map[a]
        if not os.path.exists(src):
            print(f"⚠️ skip {src}: not found")
            continue
        # skip if already did
        if os.path.exists(f"{a}_train.tfrecord") and (a=='parsed_dataset' or os.path.exists(f"Normal_{a}_train.tfrecord")):
            print(f"Skipping {a} (already TFRecord’d)")
            continue

        print(f"\n⎯⎯⎯⎯ {a} ⎯⎯⎯⎯")
        df = preprocess_windows(src)
        df_att  = df[df.label==1]
        df_norm = df[df.label==0]
        if len(df_att)>0:
            split_and_write(df_att, a)
        split_and_write(df_norm, f"Normal_{a}")

# ------------------------------------------------------------
# 3) Build train/val datasets on Normal windows
# ------------------------------------------------------------
schema = {
    'input_features': tf.io.FixedLenFeature([29*29*2], tf.int64),
    'label':          tf.io.FixedLenFeature([1],      tf.int64)
}
def parse_norm(proto):
    p = tf.io.parse_single_example(proto, schema)
    x = tf.cast(p['input_features'], tf.float32)
    return x, x

train_norm = sorted(glob.glob("Normal_*_train.tfrecord"))
val_norm   = sorted(glob.glob("Normal_*_val.tfrecord"))

# sanity check before training
if not train_norm or not val_norm:
    raise FileNotFoundError("No Normal train/val TFRecords found – cannot train!")

train_ds = (tf.data.TFRecordDataset(train_norm)
            .map(parse_norm, num_parallel_calls=tf.data.AUTOTUNE)
            .shuffle(10000).batch(64).prefetch(tf.data.AUTOTUNE))
val_ds   = (tf.data.TFRecordDataset(val_norm)
            .map(parse_norm, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(64).prefetch(tf.data.AUTOTUNE))
print(f"\n✅ train_ds files: {len(train_norm)}, val_ds files: {len(val_norm)}")

# ------------------------------------------------------------
# 4) Define & train SimpleAE
# ------------------------------------------------------------
@register_keras_serializable()
class SimpleAE(tf.keras.Model):
    def __init__(self, input_dim, latent_dim=128, **kwargs):
        super().__init__(**kwargs)
        self.input_dim  = input_dim
        self.latent_dim = latent_dim
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Dense(512),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(0.2),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(latent_dim),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(0.2),
        ])
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Dense(512),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(0.2),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(input_dim),
        ])
    def call(self, x):
        return self.decoder(self.encoder(x))
    def get_config(self):
        cfg = super().get_config()
        cfg.update({'input_dim': self.input_dim, 'latent_dim': self.latent_dim})
        return cfg
    @classmethod
    def from_config(cls, config):
        return cls(**config)

input_dim = 29*29*2
ae = SimpleAE(input_dim=input_dim, latent_dim=128)
ae.compile(optimizer=tf.keras.optimizers.Adam(1e-3,0.5,0.9), loss='mse')
ae.fit(train_ds, epochs=25, validation_data=val_ds,
       callbacks=[tf.keras.callbacks.EarlyStopping(patience=5,restore_best_weights=True)])
ae.save('simple_autoencoder.keras')

# ------------------------------------------------------------
# 5) Evaluate on Normal & Attack with ROC & Youden’s J
# ------------------------------------------------------------
loaded = tf.keras.models.load_model('simple_autoencoder.keras')

# normal validation
val_ds_norm = (
    tf.data.TFRecordDataset(val_norm)
      .map(parse_norm, num_parallel_calls=tf.data.AUTOTUNE)
      .batch(64)
)
val_x = np.concatenate([x.numpy() for x,_ in val_ds_norm], axis=0)
rec_norm = loaded.predict(val_x, verbose=0)
err_norm = np.mean((val_x - rec_norm)**2, axis=1)

# attack test
test_frs = sorted(glob.glob("*_test.tfrecord"))
attack_frs = [f for f in test_frs if not f.startswith("Normal_")]
errs = []
for af in attack_frs:
    ds = (
      tf.data.TFRecordDataset([af])
        .map(lambda p: tf.cast(tf.io.parse_single_example(p, schema)['input_features'], tf.float32),
             num_parallel_calls=tf.data.AUTOTUNE)
        .batch(64)
    )
    for xb in ds:
        rec = loaded.predict(xb, verbose=0)
        errs.append(np.mean((xb.numpy() - rec)**2, axis=1))
err_att = np.concatenate(errs)

# ROC + Youden’s J
y_true = np.concatenate([np.zeros_like(err_norm), np.ones_like(err_att)])
scores = np.concatenate([err_norm, err_att])

fpr, tpr, ths = roc_curve(y_true, scores)
roc_auc       = auc(fpr, tpr)
opt_i         = np.argmax(tpr - fpr)
opt_thresh    = ths[opt_i]

print(f"\nROC AUC = {roc_auc:.4f}")
print(f"Youden’s J optimal thresh = {opt_thresh:.6f}  (TPR={tpr[opt_i]:.3f}, FPR={fpr[opt_i]:.3f})")

y_pred = (scores > opt_thresh).astype(int)
print("\nConfusion Matrix @ optimal thresh:\n", confusion_matrix(y_true, y_pred, labels=[0,1]))
print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=['Normal','Attack']))


🔄 Rebuilding TFRecords from CSV...

⎯⎯⎯⎯ DoS ⎯⎯⎯⎯
DoS_dataset.csv: #Normal=62208, #Attack=26276


DoS: 100%|██████████| 26276/26276 [00:05<00:00, 4923.24it/s]


  → DoS_train.tfrecord: 18393 records
  → DoS_val.tfrecord: 3941 records
  → DoS_test.tfrecord: 3942 records


Normal_DoS: 100%|██████████| 62208/62208 [00:12<00:00, 4887.85it/s]


  → Normal_DoS_train.tfrecord: 43545 records
  → Normal_DoS_val.tfrecord: 9331 records
  → Normal_DoS_test.tfrecord: 9332 records

⎯⎯⎯⎯ Fuzzy ⎯⎯⎯⎯
Fuzzy_dataset.csv: #Normal=61478, #Attack=31184


...

  → Normal_gear_train.tfrecord: 43023 records
  → Normal_gear_val.tfrecord: 9219 records
  → Normal_gear_test.tfrecord: 9220 records

⎯⎯⎯⎯ parsed_dataset ⎯⎯⎯⎯
parsed_dataset.csv: #Normal=34098, #Attack=0


Normal_parsed_dataset: 100%|██████████| 34098/34098 [00:07<00:00, 4591.00it/s]


  → Normal_parsed_dataset_train.tfrecord: 23868 records
  → Normal_parsed_dataset_val.tfrecord: 5114 records
  → Normal_parsed_dataset_test.tfrecord: 5116 records

✅ train_ds files: 5, val_ds files: 5
Epoch 1/25
   3071/Unknown 20s 4ms/step - loss: 0.0363

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


3071/3071 ━━━━━━━━━━━━━━━━━━━━ 22s 5ms/step - loss: 0.0363 - val_loss: 0.0222
Epoch 2/25
3071/3071 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.0179 - val_loss: 0.0184
Epoch 3/25
3071/3071 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.0149 - val_loss: 0.0159
Epoch 4/25
3071/3071 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.0128 - val_loss: 0.0148
Epoch 5/25
3071/3071 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.0113 - val_loss: 0.0124
Epoch 6/25
3071/3071 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.0101 - val_loss: 0.0116
Epoch 7/25
3071/3071 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.0093 - val_loss: 0.0112
Epoch 8/25
3071/3071 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.0087 - val_loss: 0.0104
Epoch 9/25
3071/3071 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.0082 - val_loss: 0.0107
Epoch 10/25
3071/3071 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.0078 - val_loss: 0.0091
Epoch 11/25
3071/3071 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 0.0075 - val_loss: 0.0091
Epoch 12/25
3071/3071 ━━━━━━━━━━━━━━━━━━